## Quantium Virtual Internship - Retail Strategy and Analytics - Task 1

This Jupyter notebook follows a solution scaffold converted from R for Python for the task provided.

In [7]:
import pandas as pd

In [21]:
# Set up the dataframes
purchase_behaviour_df = pd.read_csv("C:/Users/Jacqu/OneDrive/Documents/Employment/Quantium/QuantiumVEP/Task1/QVI_purchase_behaviour.csv")
transaction_data_df = pd.read_excel("C:/Users/Jacqu/OneDrive/Documents/Employment/Quantium/QuantiumVEP/Task1/QVI_transaction_data.xlsx")

### Exploratory Data Analysis

Let's first examine the transaction data.

In [23]:
# Check the column names for each df
print(purchase_behaviour_df.columns)
print(transaction_data_df.columns)

Index(['LYLTY_CARD_NBR', 'LIFESTAGE', 'PREMIUM_CUSTOMER'], dtype='object')
Index(['DATE', 'STORE_NBR', 'LYLTY_CARD_NBR', 'TXN_ID', 'PROD_NBR',
       'PROD_NAME', 'PROD_QTY', 'TOT_SALES'],
      dtype='object')


In [28]:
# Check if the columns that are expected to be numeric are in numeric form
print("Purchase Behvaiour Datatypes: \n", purchase_behaviour_df.dtypes, "\n")
print("Transaction Data Datatypes: \n", transaction_data_df.dtypes)

Purchase Behvaiour Datatypes: 
 LYLTY_CARD_NBR       int64
LIFESTAGE           object
PREMIUM_CUSTOMER    object
dtype: object 

Transaction Data Datatypes: 
 DATE                int64
STORE_NBR           int64
LYLTY_CARD_NBR      int64
TXN_ID              int64
PROD_NBR            int64
PROD_NAME          object
PROD_QTY            int64
TOT_SALES         float64
dtype: object


We know that if there is mixed datatypes, it will get stored as an object dtype. Thus, the numeric columns are all satisfactory, since they all int64 and float64 dtypes.

In [ ]:
# Check if the date columns are in date format
